In [1]:
import tensorflow as tf
import numpy as np

import pickle
import time

In [2]:
checkpoint_dir = "F:/Python/final/test/"

In [ ]:
#下面是保存tf创建一个ckpt文件的过程

In [3]:
#定义3个varible
with tf.variable_scope("foo",reuse=False):
    v1 = tf.get_variable(name="v1",shape=[3,3],dtype=tf.float32,initializer=tf.zeros_initializer())
    v2 = tf.get_variable(name="v2",shape=[3,3],dtype=tf.float32,initializer=tf.random_normal_initializer(seed=1),trainable=False)
    v3 = v1 + v2

In [6]:
#注意，这里v1和v3是不一样的。v1作为variable不是tensor类型。但是平时用的时候v1也是tensor。
#这里要打个问号
#而v3作为一个运算肯定是tensor
print(type(v1))
print(v1)
print(type(v3))
print(v3)

<tf.Variable 'foo/v1:0' shape=(3, 3) dtype=float32_ref> <class 'tensorflow.python.ops.variables.Variable'>
Tensor("foo/add:0", shape=(3, 3), dtype=float32) <class 'tensorflow.python.framework.ops.Tensor'>


In [4]:
#定义一个saver，还有一些参数，具体查看文档
saver = tf.train.Saver(max_to_keep=2)
#下面这句是为了生成meta文件的json格式文件，平时可以注释掉
# saver.export_meta_graph(checkpoint_dir + "mode.ckpt.meda.json", as_text=True)
#这里是把3个tensor初始化然后保存起来
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(v1))
    print(sess.run(v2))
    sess.run(v3)
    #如果有参数global_step=10，那么最后的文件名会变成xx/model.ckpt-10
    saver.save(sess,checkpoint_dir + "model.ckpt",global_step=10)

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]
 [ 0.59282297 -2.12292957 -0.72289723]]


In [ ]:
#下面是恢复参数的一种方法，这种方法需要和原本一样重新定义varible

In [ ]:
#定义3个varible
with tf.variable_scope("foo",reuse=False):
    v1 = tf.get_variable(name="v1",shape=[3,3],dtype=tf.float32,initializer=tf.zeros_initializer())
    v2 = tf.get_variable(name="v2",shape=[3,3],dtype=tf.float32,initializer=tf.random_normal_initializer(seed=1),trainable=False)
    v3 = v1 + v2

In [5]:
#读取ckpt里保存的参数
sess = tf.InteractiveSession()
saver = tf.train.Saver(max_to_keep=2)
#如果有checkpoint这个文件可以加下面这句话，如果只有一个ckpt文件就不需要这个if
if tf.train.get_checkpoint_state(checkpoint_dir): #确认是否存在
    saver.restore(sess,checkpoint_dir + "model.ckpt-10")
    print("load ok!")
else:
    print("ckpt文件不存在")

INFO:tensorflow:Restoring parameters from C:/Users/hsqyc/Desktop/model.ckpt-10
load ok!


In [7]:
#这里可以显示v3这个tensor是有值的
print(sess.run(v3))

[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]
 [ 0.59282297 -2.12292957 -0.72289723]]


In [ ]:
#下面是恢复varible的另外一种方法，不需要定义，直接从meta文件里读(meta文件里存放的是元图数据)

In [3]:
saver = tf.train.import_meta_graph(checkpoint_dir + "model.ckpt-10.meta")
# sess = tf.InteractiveSession()
with tf.Session() as sess:
    saver.restore(sess,checkpoint_dir + "model.ckpt-10")
    #根据变量名来直接获取tensor的值，而不需要重新定义tensor
    print(sess.run(tf.get_default_graph().get_tensor_by_name("foo/v2:0")))

INFO:tensorflow:Restoring parameters from F:/Python/final/test/model.ckpt-10
[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]
 [ 0.59282297 -2.12292957 -0.72289723]]


In [ ]:
#下面是有选择的从ckpt中恢复部分变量

In [3]:
#只定义1个varible（这里定义的名字要跟原来一样）
with tf.variable_scope("foo",reuse=False):
    x1 = tf.get_variable(name="v1",shape=[3,3],dtype=tf.float32,initializer=tf.zeros_initializer())
#     x2 = tf.get_variable(name="v2",shape=[3,3],dtype=tf.float32,initializer=tf.random_normal_initializer(seed=1),trainable=False)
#     x3 = x1 + x2

In [5]:
#然后定义一个list，包含需要加载的varible
A = [x1]
#读取ckpt里保存的参数
# sess = tf.InteractiveSession()
with tf.Session() as sess:
    #构建saver的时候添加参数A
    saver = tf.train.Saver(max_to_keep=2,var_list=A)
    if tf.train.get_checkpoint_state(checkpoint_dir): #确认是否存在
        saver.restore(sess,checkpoint_dir + "model.ckpt-10")
        print("load ok!")
    else:
        print("ckpt文件不存在")
    print(sess.run(x1))

INFO:tensorflow:Restoring parameters from F:/Python/final/test/model.ckpt-10
load ok!
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


In [ ]:
#下面是从重新定义原来一样的变量，但是变量名不一样。然后从ckpt中恢复值到新变量

In [3]:
#定义3个不同名字的tensor
with tf.variable_scope("foo",reuse=False):
    x1 = tf.get_variable(name="other-v1",shape=[3,3],dtype=tf.float32,initializer=tf.zeros_initializer())
    x2 = tf.get_variable(name="other-v2",shape=[3,3],dtype=tf.float32,initializer=tf.random_normal_initializer(seed=1),trainable=False)
    x3 = x1 + x2

In [4]:
#然后定义一个字典，用法和上面一样。字典的key是旧名字，值是新的引用
dict = {"foo/v1":x1,"foo/v2":x2}
#读取ckpt里保存的参数
# sess = tf.InteractiveSession()
with tf.Session() as sess:
    saver = tf.train.Saver(max_to_keep=2,var_list=dict)
    if tf.train.get_checkpoint_state(checkpoint_dir): #确认是否存在
        saver.restore(sess,checkpoint_dir + "model.ckpt-10")
        print("load ok!")
        print(x2)
        print(sess.run(x2))
    else:
        print("ckpt文件不存在")
    

INFO:tensorflow:Restoring parameters from F:/Python/final/test/model.ckpt-10
load ok!
<tf.Variable 'foo/other-v2:0' shape=(3, 3) dtype=float32_ref>
[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]
 [ 0.59282297 -2.12292957 -0.72289723]]


In [ ]:
#下面是几种在tf中查看变量的方法

In [11]:
#直接用tf的函数读取变量
print(tf.global_variables())
print(tf.trainable_variables())

[<tf.Variable 'foo/other-v1:0' shape=(3, 3) dtype=float32_ref>, <tf.Variable 'foo/other-v2:0' shape=(3, 3) dtype=float32_ref>]
[<tf.Variable 'foo/other-v1:0' shape=(3, 3) dtype=float32_ref>]


In [16]:
#tl.layers中也有函数可以读变量，而且还比较方便
#其实就是tf.global_variables()的封装
from tensorlayer.layers import *
print_all_variables()

  [*] printing global variables
  var   0: (3, 3)            foo/other-v1:0
  var   1: (3, 3)            foo/other-v2:0


In [ ]:
#下面是从ckpt中直接读取保存的变量名及其值

In [10]:
#创建一个读写器
reader = tf.train.NewCheckpointReader(checkpoint_dir + "model.ckpt-10")
#读变量和shape的映射字典
all_variables_shape = reader.get_variable_to_shape_map()
#读变量和dtyte的映射字典
all_variables_dtyte = reader.get_variable_to_dtype_map()

print(all_variables_shape)
print(all_variables_dtyte)
#从读写器中根据变量名获取变量的值
print(reader.get_tensor("foo/v2") )

{'foo/v1': [3, 3], 'foo/v2': [3, 3]}
{'foo/v1': tf.float32, 'foo/v2': tf.float32}
[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]
 [ 0.59282297 -2.12292957 -0.72289723]]
